In [ ]:
using Plots, Interact
gr(yflip = true,          # matrix convention that rows are numbered from the top
    color = :grays,
    legend = :none)

In [ ]:
# extend conv2 to accept "full" or "valid" as an third argument
function Base.conv2(A, B, shape::String)   
    if shape == "full"
        return Base.conv2(A,B)
    elseif shape == "valid"
        ranges = [ min(a,b):max(a,b) for (a,b) in zip(size(A),size(B)) ]
        return Base.conv2(A,B)[ranges...]
    else
        error("shape must be full or valid")
    end
end

In [ ]:
# function for displaying a stack of images
# imgstack is mxnxp array that contains p images, each of which is mxn 
function montage(imgstack)
    plot(
        [heatmap(imgstack[:,:,i]) for i=1:size(imgstack,3)]...,
        legend=:none, axis=nothing     # options necessary to get nice spacing of the images
    )
end

In [ ]:
# define four oriented bar detectors
# note that these have inhibitory backgrounds (-1)
nfeature = 4
w = zeros(3,3,nfeature)
w[:,:,1] = [-1 -1 -1; 1 1 1; -1 -1 -1]  # 0 deg  (horizontal)
w[:,:,3] = w[:,:,1]'                    # 90 deg  (vertical)
w[:,:,4] = 2*eye(3)-1                   # 135 deg
w[:,:,2] = w[:,end:-1:1,4]              # 45 deg
w

In [ ]:
montage(w)

In [ ]:
two = [
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0;
 0  0  0  0  0  0  1  0  0  0  1  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0;
 0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0;
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  1  1  1  1  1  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
]
heatmap(two, axis=nothing)

In [ ]:
x1 = zeros(14,14,nfeature)
for ifeature=1:nfeature
      x1[:,:,ifeature] = conv2(w[:,:,ifeature],float(two),"valid") .> 2.5;   # high thres: conjunction
end
plot(montage(w), heatmap(two), montage(x1),axis=nothing)

In [ ]:
twodistort = [
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  1  1  1  1  0  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0;
 0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0;
 0  0  0  0  0  0  0  1  1  1  0  0  0  0  0  0;
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  1  1  1  1  1  1  1  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0;
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
]
plot(heatmap(two), heatmap(twodistort),axis=nothing)

In [ ]:
X0 = [two, twodistort]
@manipulate for i=1:2; 
    heatmap(X0[i],axis=nothing)
end

In [ ]:
x1distort = zeros(14,14,nfeature)
for ifeature=1:nfeature
      x1distort[:,:,ifeature] = conv2(w[:,:,ifeature],float(twodistort),"valid") .> 2.5;   # high thres: conjunction
end

In [ ]:
X1 = [x1, x1distort]
@manipulate for i=1:2; 
    plot(heatmap(X0[i],axis=nothing),montage(X1[i]))
end

In [ ]:
function sumpool(img)
    # sum in nonoverlapping 2x2 blocks
    box = [1.0 1.0; 1.0 1.0]
    float(conv2(box,img,"valid")[1:2:end,1:2:end] .> 0.5)
end

In [ ]:
x2 = zeros(7,7,4)
for k=1:4
    x2[:,:,k] = sumpool(x1[:,:,k])
end
x2distort = zeros(7,7,4)
for k=1:4
    x2distort[:,:,k] = sumpool(x1distort[:,:,k])
end
X2 = [x2, x2distort]
@manipulate for i=1:2; 
    plot(heatmap(X0[i],axis=nothing),montage(X1[i]),montage(X2[i]))
end

In [ ]:
w3 = zeros(2,2,4)
w3[:,:,1] = [
    -1. -1.;
    1. -1.
]
w3[:,:,2] = [
    -1. 1.; 
    -1. -1.
]
montage(w3)

In [ ]:
x3 = zeros(6,6)
for ifeature = 1:nfeature
    x3 += conv2(w3[:,:,ifeature],x2[:,:,ifeature],"valid")
end
x3 = float(x3 .> 1.5)
montage(x3)